In [1]:
!pip install scipy==1.1.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached scipy-1.1.0-cp38-cp38-linux_x86_64.whl
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.19.2 requires scipy>=1.4.1, but you have scipy 1.1.0 which is incompatible.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the batch_size here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/datasets/Project_data/val.csv').readlines())

In [5]:
len(train_doc)

663

In [6]:
len(val_doc)

100

In [7]:
batch_size = 10 #experiment with the batch size

#PreProcessing

In [8]:
# Helper Functions and our current preprogessing pipeline
# pre process ===> conbert_to_grayscale --> normalize_data

# Convert to grayscale
def convert_to_grayscale(data):
    return data.mean(axis=-1,keepdims=1) 

def normalize_data(data):
    return data/127.5-1

def preprocess_pipeline(data):
    return normalize_data(convert_to_grayscale(data))

#Generator

This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with img_idx, y,z and normalization such that you get high accuracy.

In [9]:
def generator(source_path, folder_list, batch_size):
    x = 30 # number of frames
    y = 120 # image width
    z = 120 # image height

    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = normalize_data(temp)

                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,c)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = normalize_data(temp)
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels 
        

In [10]:
curr_dt_time = datetime.datetime.now()
train_path = 'datasets/Project_data/train'
val_path = 'datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [11]:
c = 3

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
input_shape = (30, 120, 120, c)

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [12]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In [13]:


#write your model here

model_a = Sequential()

model_a.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(nb_dense[0], activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(nb_dense[1], activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(nb_dense[2], activation='softmax'))


2022-05-18 13:53:31.812043: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-05-18 13:53:31.812106: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12104 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:88:00.0, compute capability: 8.6


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [14]:
model_a.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization (BatchN  (None, 30, 120, 120, 8)  32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 15, 60, 60, 8)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_1 (Batc  (None, 15, 60, 60, 16)   6

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [15]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [16]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the Reducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [18]:
steps_per_epoch

67

In [19]:
validation_steps

10

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [20]:
model_a.fit_generator(train_generator
                      , steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                callbacks=callbacks_list, validation_data=val_generator, 
                validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_389/3707964710.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_a.fit_generator(train_generator
/tmp/ipykernel_389/3762286381.py:18: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  datasets/Project_data/train ; batch size = 10


/tmp/ipykernel_389/3762286381.py:22: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
  temp = imresize(image,(120,120))


Epoch 1/10


2022-05-18 13:53:34.814203: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302
2022-05-18 13:53:36.674983: I tensorflow/stream_executor/cuda/cuda_blas.cc:1792] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


66/67 [============================>.] - ETA: 1s - loss: 4.9773 - categorical_accuracy: 0.2515

/tmp/ipykernel_389/3762286381.py:44: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
/tmp/ipykernel_389/3762286381.py:48: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
  temp = imresize(image,(120,120))


67/67 [==============================] - ETA: 0s - loss: 4.9657 - categorical_accuracy: 0.2504Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1813_53_31.147025/model-00001-4.96570-0.25038-1.85923-0.32000.h5
67/67 [==============================] - 90s 1s/step - loss: 4.9657 - categorical_accuracy: 0.2504 - val_loss: 1.8592 - val_categorical_accuracy: 0.3200 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.2392 - categorical_accuracy: 0.2488
Epoch 00002: saving model to model_init_2022-05-1813_53_31.147025/model-00002-2.23923-0.24876-1.33031-0.47000.h5
67/67 [==============================] - 35s 531ms/step - loss: 2.2392 - categorical_accuracy: 0.2488 - val_loss: 1.3303 - val_categorical_accuracy: 0.4700 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.8633 - categorical_accuracy: 0.2886
Epoch 00003: saving model to model_init_2022-05-1813_53_31.147025/model-0

In [21]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

In [22]:
def new_generator(source_path, folder_list, batch_size):
      
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,nb_frames)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(nb_rows,nb_cols))
                    #temp = normalize_data(temp)
                    temp = preprocess_pipeline(temp)
                    
                    batch_data[folder,idx] = temp

       
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(nb_rows,nb_cols))
                    #temp = normalize_data(temp)
                    temp = preprocess_pipeline(temp)
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
        
  


In [23]:
from keras.losses import categorical_crossentropy

# Define model
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(nb_frames,nb_rows,nb_cols,nb_channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(nb_classes, activation='softmax'))

model_b.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 120, 120, 32)  2624      
                                                                 
 activation_4 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 conv3d_5 (Conv3D)           (None, 30, 120, 120, 32)  27680     
                                                                 
 activation_5 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 10, 40, 40, 32)   0         
 3D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 10, 40, 40, 32)    0         
                                                      

In [24]:
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)

In [25]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10


/tmp/ipykernel_389/3161962229.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/tmp/ipykernel_389/455585104.py:14: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
/tmp/ipykernel_389/455585104.py:18: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
  temp = imresize(image,(nb_rows,nb_cols))


Epoch 1/10
66/67 [============================>.] - ETA: 1s - loss: 2.0038 - categorical_accuracy: 0.2061

/tmp/ipykernel_389/455585104.py:37: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
/tmp/ipykernel_389/455585104.py:41: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
  temp = imresize(image,(nb_rows,nb_cols))


67/67 [==============================] - ETA: 0s - loss: 2.0021 - categorical_accuracy: 0.2051Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1813_53_31.147025/model-00001-2.00211-0.20513-1.60898-0.23000.h5
67/67 [==============================] - 95s 1s/step - loss: 2.0021 - categorical_accuracy: 0.2051 - val_loss: 1.6090 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6108 - categorical_accuracy: 0.1940
Epoch 00002: saving model to model_init_2022-05-1813_53_31.147025/model-00002-1.61085-0.19403-1.60907-0.17000.h5
67/67 [==============================] - 38s 560ms/step - loss: 1.6108 - categorical_accuracy: 0.1940 - val_loss: 1.6091 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6105 - categorical_accuracy: 0.1990
Epoch 00003: saving model to model_init_2022-05-1813_53_31.147025/model-0

In [26]:
# Define model
model_c = Sequential()
model_c.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(nb_frames,nb_rows,nb_cols,nb_channel), padding="same"))
model_c.add(Activation('relu'))
model_c.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Flatten())
model_c.add(Dense(512, activation='relu'))
model_c.add(BatchNormalization())
model_c.add(Dropout(0.5))
model_c.add(Dense(nb_classes, activation='softmax'))
model_c.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_c.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 120, 120, 32)  2624      
                                                                 
 activation_8 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 120, 120, 32)  27680     
                                                                 
 activation_9 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 10, 40, 40, 32)   0         
 3D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 10, 40, 40, 32)    0         
                                                      

In [27]:
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)

In [28]:
num_epochs = 20
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10


/tmp/ipykernel_389/191411352.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/tmp/ipykernel_389/455585104.py:14: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
/tmp/ipykernel_389/455585104.py:18: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
  temp = imresize(image,(nb_rows,nb_cols))


Epoch 1/20
66/67 [============================>.] - ETA: 1s - loss: 1.9870 - categorical_accuracy: 0.3167

/tmp/ipykernel_389/455585104.py:37: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
/tmp/ipykernel_389/455585104.py:41: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
  temp = imresize(image,(nb_rows,nb_cols))


67/67 [==============================] - ETA: 0s - loss: 1.9911 - categorical_accuracy: 0.3152Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1813_53_31.147025/model-00001-1.99112-0.31523-6.28247-0.18000.h5
67/67 [==============================] - 91s 1s/step - loss: 1.9911 - categorical_accuracy: 0.3152 - val_loss: 6.2825 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 2.0446 - categorical_accuracy: 0.2736
Epoch 00002: saving model to model_init_2022-05-1813_53_31.147025/model-00002-2.04461-0.27363-2.72361-0.37000.h5
67/67 [==============================] - 38s 569ms/step - loss: 2.0446 - categorical_accuracy: 0.2736 - val_loss: 2.7236 - val_categorical_accuracy: 0.3700 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.9079 - categorical_accuracy: 0.2537
Epoch 00003: saving model to model_init_2022-05-1813_53_31.147025/model-0

In [29]:
nb_frames = 30 # number of frames
nb_rows = 64 # image width
nb_cols = 64 # image height 

nb_classes = 5
nb_channel = 1

In [30]:
# Define model
model_d = Sequential()
model_d.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(nb_frames,nb_rows,nb_cols,nb_channel), padding="same"))
model_d.add(Activation('relu'))
model_d.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_d.add(Activation('relu'))
model_d.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_d.add(Dropout(0.25))

model_d.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_d.add(Activation('relu'))
model_d.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_d.add(Activation('relu'))
model_d.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_d.add(Dropout(0.25))

model_d.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_d.add(Activation('relu'))
model_d.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_d.add(Activation('relu'))
model_d.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_d.add(Dropout(0.25))

model_d.add(Flatten())
model_d.add(Dense(512, activation='relu'))
model_d.add(BatchNormalization())
model_d.add(Dropout(0.5))
model_d.add(Dense(nb_classes, activation='softmax'))
model_d.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_d.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_14 (Conv3D)          (None, 30, 64, 64, 32)    896       
                                                                 
 activation_14 (Activation)  (None, 30, 64, 64, 32)    0         
                                                                 
 conv3d_15 (Conv3D)          (None, 30, 64, 64, 32)    27680     
                                                                 
 activation_15 (Activation)  (None, 30, 64, 64, 32)    0         
                                                                 
 max_pooling3d_9 (MaxPooling  (None, 10, 22, 22, 32)   0         
 3D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 10, 22, 22, 32)    0         
                                                      

In [31]:
batch_size = 5
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)
num_epochs = 20
model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)


Source path =  datasets/Project_data/train ; batch size = 5


/tmp/ipykernel_389/3837395513.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/tmp/ipykernel_389/455585104.py:14: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
/tmp/ipykernel_389/455585104.py:18: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
  temp = imresize(image,(nb_rows,nb_cols))


Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 2.4010 - categorical_accuracy: 0.1940Source path =  datasets/Project_data/val ; batch size = 5

Epoch 00001: saving model to model_init_2022-05-1813_53_31.147025/model-00001-2.40105-0.19403-9.20659-0.18000.h5
67/67 [==============================] - 38s 566ms/step - loss: 2.4010 - categorical_accuracy: 0.1940 - val_loss: 9.2066 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 2/20
65/67 [============================>.] - ETA: 0s - loss: 2.1799 - categorical_accuracy: 0.2277

/tmp/ipykernel_389/455585104.py:37: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
/tmp/ipykernel_389/455585104.py:41: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
  temp = imresize(image,(nb_rows,nb_cols))


67/67 [==============================] - ETA: 0s - loss: 2.1895 - categorical_accuracy: 0.2236
Epoch 00002: saving model to model_init_2022-05-1813_53_31.147025/model-00002-2.18946-0.22356-2.94010-0.20000.h5
67/67 [==============================] - 37s 563ms/step - loss: 2.1895 - categorical_accuracy: 0.2236 - val_loss: 2.9401 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 2.1108 - categorical_accuracy: 0.2239
Epoch 00003: saving model to model_init_2022-05-1813_53_31.147025/model-00003-2.11078-0.22388-8.18307-0.10000.h5
67/67 [==============================] - 24s 360ms/step - loss: 2.1108 - categorical_accuracy: 0.2239 - val_loss: 8.1831 - val_categorical_accuracy: 0.1000 - lr: 0.0010
Epoch 4/20
67/67 [==============================] - ETA: 0s - loss: 2.2047 - categorical_accuracy: 0.1791
Epoch 00004: saving model to model_init_2022-05-1813_53_31.147025/model-00004-2.20475-0.17910-2.96408-0.18000.h5

Epoch 00004: Re

In [32]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 1

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model_e = Sequential()

model_e.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_e.add(BatchNormalization())
model_e.add(Activation('relu'))

model_e.add(MaxPooling3D(pool_size=(2,2,2)))

model_e.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_e.add(BatchNormalization())
model_e.add(Activation('relu'))

model_e.add(MaxPooling3D(pool_size=(2,2,2)))

model_e.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_e.add(BatchNormalization())
model_e.add(Activation('relu'))

model_e.add(MaxPooling3D(pool_size=(2,2,2)))

model_e.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_e.add(BatchNormalization())
model_e.add(Activation('relu'))

model_e.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_e.add(Flatten())

model_e.add(Dense(nb_dense[0], activation='relu'))
model_e.add(Dropout(0.5))

model_e.add(Dense(nb_dense[1], activation='relu'))
model_e.add(Dropout(0.5))

#softmax layer
model_e.add(Dense(nb_dense[2], activation='softmax'))
model_e.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_e.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_20 (Conv3D)          (None, 30, 120, 120, 8)   224       
                                                                 
 batch_normalization_6 (Batc  (None, 30, 120, 120, 8)  32        
 hNormalization)                                                 
                                                                 
 activation_20 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_12 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_21 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_7 (Batc  (None, 15, 60, 60, 16)  

In [33]:
batch_size = 10
train_generator = new_generator(train_path, train_doc, batch_size)
val_generator = new_generator(val_path, val_doc, batch_size)
num_epochs = 30
model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10


/tmp/ipykernel_389/981344006.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/tmp/ipykernel_389/455585104.py:14: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
/tmp/ipykernel_389/455585104.py:18: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
  temp = imresize(image,(nb_rows,nb_cols))


Epoch 1/30
66/67 [============================>.] - ETA: 1s - loss: 4.7305 - categorical_accuracy: 0.3030

/tmp/ipykernel_389/455585104.py:37: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
/tmp/ipykernel_389/455585104.py:41: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
  temp = imresize(image,(nb_rows,nb_cols))


67/67 [==============================] - ETA: 0s - loss: 4.7163 - categorical_accuracy: 0.3017Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1813_53_31.147025/model-00001-4.71631-0.30166-2.02713-0.17000.h5
67/67 [==============================] - 81s 1s/step - loss: 4.7163 - categorical_accuracy: 0.3017 - val_loss: 2.0271 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 2/30
67/67 [==============================] - ETA: 0s - loss: 2.4562 - categorical_accuracy: 0.3085
Epoch 00002: saving model to model_init_2022-05-1813_53_31.147025/model-00002-2.45616-0.30846-1.58276-0.23000.h5
67/67 [==============================] - 34s 508ms/step - loss: 2.4562 - categorical_accuracy: 0.3085 - val_loss: 1.5828 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 3/30
67/67 [==============================] - ETA: 0s - loss: 1.8004 - categorical_accuracy: 0.2836
Epoch 00003: saving model to model_init_2022-05-1813_53_31.147025/model-0

In [34]:
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [35]:
def aug_generator(source_path, folder_list, batch_size):
      
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,nb_frames)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
  
            batch_data_aug2 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug2 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
 
            batch_data_aug3 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug3 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
      
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                M2 = get_random_affine()
                M3 = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes   
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    batch_data_aug2[folder,idx] = cv2.warpAffine(resized, M2, (resized.shape[0], resized.shape[1]))
                    batch_data_aug3[folder,idx] = cv2.warpAffine(resized, M3, (resized.shape[0], resized.shape[1]))
                        
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug2[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug3[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug2, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug3, axis = 0)
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug2, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug3, axis = 0)
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
  
            batch_data_aug2 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug2 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
   
            batch_data_aug3 = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug3 = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
      
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                M2 = get_random_affine()
                M3 = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    batch_data_aug2[folder,idx] = cv2.warpAffine(resized, M2, (resized.shape[0], resized.shape[1]))
                    batch_data_aug3[folder,idx] = cv2.warpAffine(resized, M3, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug2[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug3[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug2, axis = 0) 
            batch_data = np.append(batch_data, batch_data_aug3, axis = 0)
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug2, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug3, axis = 0) 
            yield batch_data, batch_labels

In [36]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model_f = Sequential()

model_f.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_f.add(Flatten())

model_f.add(Dense(nb_dense[0], activation='relu'))
model_f.add(Dropout(0.5))

model_f.add(Dense(nb_dense[1], activation='relu'))
model_f.add(Dropout(0.5))

#softmax layer
model_f.add(Dense(nb_dense[2], activation='softmax'))
model_f.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_f.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_24 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_10 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_24 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_16 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_25 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_11 (Bat  (None, 15, 60, 60, 16)  

In [37]:
batch_size = 5
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)


Source path =  datasets/Project_data/train ; batch size = 5


/tmp/ipykernel_389/1853878729.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 3.5132 - categorical_accuracy: 0.2821Source path =  datasets/Project_data/val ; batch size = 5

Epoch 00001: saving model to model_init_2022-05-1813_53_31.147025/model-00001-3.51316-0.28209-4.75352-0.22000.h5
67/67 [==============================] - 56s 828ms/step - loss: 3.5132 - categorical_accuracy: 0.2821 - val_loss: 4.7535 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.4473 - categorical_accuracy: 0.3807
Epoch 00002: saving model to model_init_2022-05-1813_53_31.147025/model-00002-1.44726-0.38066-1.67812-0.38000.h5
67/67 [==============================] - 57s 860ms/step - loss: 1.4473 - categorical_accuracy: 0.3807 - val_loss: 1.6781 - val_categorical_accuracy: 0.3800 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.5458 - categorical_accuracy: 0.3358
Epoch 00003: saving model to model_init_2022-05-1813_53_31.1

In [38]:
def aug_generator(source_path, folder_list, batch_size):
      
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,nb_frames)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
  
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes   
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            
            batch_data = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            batch_data_aug = np.zeros((batch_size,nb_frames,nb_rows,nb_cols,nb_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels_aug = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                M = get_random_affine()
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
                    batch_data[folder,idx] = resized
                    batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                batch_labels_aug[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 

            yield batch_data, batch_labels
        

In [39]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model_g = Sequential()

model_g.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_g.add(Flatten())

model_g.add(Dense(nb_dense[0], activation='relu'))
model_g.add(Dropout(0.3))

model_g.add(Dense(nb_dense[1], activation='relu'))
model_g.add(Dropout(0.3))

#softmax layer
model_g.add(Dense(nb_dense[2], activation='softmax'))
model_g.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_g.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_28 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_14 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_28 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_20 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_29 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_15 (Bat  (None, 15, 60, 60, 16)  

In [40]:
batch_size = 10
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10


/tmp/ipykernel_389/298245867.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 2.7963 - categorical_accuracy: 0.3243Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1813_53_31.147025/model-00001-2.79627-0.32428-5.19403-0.28500.h5
67/67 [==============================] - 69s 1s/step - loss: 2.7963 - categorical_accuracy: 0.3243 - val_loss: 5.1940 - val_categorical_accuracy: 0.2850 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.6738 - categorical_accuracy: 0.2910
Epoch 00002: saving model to model_init_2022-05-1813_53_31.147025/model-00002-1.67376-0.29104-4.12701-0.23000.h5
67/67 [==============================] - 27s 403ms/step - loss: 1.6738 - categorical_accuracy: 0.2910 - val_loss: 4.1270 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.5436 - categorical_accuracy: 0.3856
Epoch 00003: saving model to model_init_2022-05-1813_53_31.147

In [41]:
nb_frames = 30 # number of frames
nb_rows = 120 # image width
nb_cols = 120 # image height 

nb_classes = 5
nb_channel = 3

input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model_h = Sequential()

model_h.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(Activation('relu'))
model_h.add(Dropout(0.25))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_h.add(Flatten())

model_h.add(Dense(nb_dense[0], activation='relu'))
model_h.add(Dropout(0.5))

model_h.add(Dense(nb_dense[1], activation='relu'))
model_h.add(Dropout(0.5))

#softmax layer
model_h.add(Dense(nb_dense[2], activation='softmax'))
model_h.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_h.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_32 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_18 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_32 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_24 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_33 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_19 (Bat  (None, 15, 60, 60, 16)  

In [42]:
batch_size = 10
train_generator = aug_generator(train_path, train_doc, batch_size)
val_generator = aug_generator(val_path, val_doc, batch_size)
num_epochs = 20
model_h.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)


Source path =  datasets/Project_data/train ; batch size = 10


/tmp/ipykernel_389/320957926.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_h.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 1.8155 - categorical_accuracy: 0.2843Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-05-1813_53_31.147025/model-00001-1.81550-0.28431-2.73964-0.28000.h5
67/67 [==============================] - 68s 1s/step - loss: 1.8155 - categorical_accuracy: 0.2843 - val_loss: 2.7396 - val_categorical_accuracy: 0.2800 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.4600 - categorical_accuracy: 0.3856
Epoch 00002: saving model to model_init_2022-05-1813_53_31.147025/model-00002-1.45996-0.38557-1.37664-0.44000.h5
67/67 [==============================] - 28s 417ms/step - loss: 1.4600 - categorical_accuracy: 0.3856 - val_loss: 1.3766 - val_categorical_accuracy: 0.4400 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.4475 - categorical_accuracy: 0.3607
Epoch 00003: saving model to model_init_2022-05-1813_53_31.147